In [ ]:
### calculate daily average air temperature from hourly average and place onto Will's 160x160 grid

In [14]:
import numpy as np
from scipy.stats import pearsonr
import datetime
import pandas as pd
import glob
import matplotlib.pyplot as plt
from matplotlib import gridspec
import datetime
from regrid import regrid
import seaborn as sns
import os
import tqdm
from netCDF4 import Dataset
import warnings
warnings.filterwarnings("ignore")

In [58]:
# creating numpy are with daily temperature data on WG grid for 2010-2020

datapath = '/Users/carmennab/PhD/OI_PolarSnow/ERA5/temp/'
lats = np.load('/Users/carmennab/PhD/OI_PolarSnow/freeboard_daily_processed/Robbie_lat.npy')
lons = np.load('/Users/carmennab/PhD/OI_PolarSnow/freeboard_daily_processed/Robbie_lon.npy')
era5_lon,era5_lat = np.meshgrid(np.array(lon),np.array(lat))
missing = np.full((153,160,160),np.nan)
years = ['2010', '2011', '2012','2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

all_temps = {}

for year in tqdm.tqdm(years):
    
    # load ERA5 temperature data
    file = datapath + year +'.nc'
    data = Dataset(file,'r')
    lon = data['longitude']
    lat = data['latitude']
    temp = data['t2m']
    #hours = data['time']    
           
    # turn hourly data into daily data and regrid onto WG (160x160) grid        
    temp_daily = []
    #dates_daily = []

    for i in np.arange(0,temp.shape[0],24):

        #dates_daily.append(dates[i].strftime('%Y%m%d'))

        temps_on_day = temp[i:i+24]

        day_ave = np.nanmean(temps_on_day,axis=0)

        regrid_ave = regrid(day_ave.ravel(), era5_lon.ravel(), era5_lat.ravel(), lons, lats,'linear')

        temp_daily.append(regrid_ave)
    
    # create full year list, with NaNs for days in May - September
    year_temps = np.concatenate((temp_daily[:-92],missing,temp_daily[-92:]),axis=0)

    all_temps[year] = year_temps


100%|████████████████████████████████████████| 11/11 [1:22:48<00:00, 451.70s/it]


In [59]:
# create one array containing data for all years and save
all_temps_arr = np.concatenate((all_temps['2010'], all_temps['2011'], all_temps['2012'], all_temps['2013'], 
                               all_temps['2014'], all_temps['2015'], all_temps['2016'], all_temps['2017'], 
                               all_temps['2018'], all_temps['2019'], all_temps['2020']),axis=0)
# np.save('/Users/carmennab/PhD/OI_PolarSnow/ERA5/temp/2010-2020.npy',all_temps_arr)